## This notebook is about data preparation to make a NeuralNetwork to analyze some metallglasses. we start by reading atoms positions from the lapmmps output file. then we make seprate files for each step in lammps output. we use no header files to calculate voroni analyizes of the atoms using voro++ library. then we make a whole dataframe with atoms, their positions and their 20-nearest-neighbours. ##

### Import liberies and os ###

In [1]:
import numpy as np
import pandas as pd
import os

### function which is used in calcualting distances for first 20-nearest-neighbours ###

In [2]:
def calculate_distance(x1, y1, z1, x2, y2, z2):
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2 + (z1 - z2) ** 2) ** 0.5, x2, y2, z2


### function which is used in voro++ library to make a file with .vol format ###

In [3]:
def run_voro(filename, xmin, xmax, ymin, ymax, zmin, zmax):
    VORO_INPUT_PATH = 'Lammps_Steps_No_Header/no_header_' + filename
    voro_command = f'voro++ -c "%i %s %A %v" -o -v -r {xmin - 1} {float(xmax + 1)} {ymin - 1} {float(ymax + 1)} {zmin - 1} {float(zmax + 1)} {VORO_INPUT_PATH}'
    os.system(voro_command)

In [4]:
ATOM_NUMBER_IN_EACH_CELL=13500
DEATIL_LINES=9
CU_RADIUS = 1.28
ZR_RADIUS = 1.60

### Below cell seprate each step from the main lammps output. each file has (atom number in each cell + detail) lines ###

In [31]:
lines_per_file = ATOM_NUMBER_IN_EACH_CELL + DEATIL_LINES
smallfile = None
file_counter = 1
with open('Data/LAMMPS_OUTPUT.lammpsTrj') as bigfile:
    for lineno, line in enumerate(bigfile):
        if lineno % lines_per_file == 0:
            if smallfile:
                smallfile.close()
            small_filename = 'Lammps_Step_{}.txt'.format(file_counter)
            file_counter = file_counter + 1
            smallfile = open('Lammps_Steps/' + small_filename, "w")
        smallfile.write(line)
    if smallfile:
        smallfile.close()

### below cell is about removing first 9 lines from every lammp_step file and save it in anoter directory called Lammps_Steps_No_Header ###

In [9]:
lammps_directory = 'Lammps_Steps'
lammps_no_header_directory = 'Lammps_Steps_No_Header'
for filename in os.listdir(lammps_directory):
    if filename.startswith('Lammps'):
        with open(os.path.join(lammps_directory, filename)) as f:
            lammps_out_file = f.readlines()
            lammps_out_file = lammps_out_file[9:]
            lammps_out_file_final = []
            for l in lammps_out_file:
                l_parts = l.split(" ")
                l_parts = l_parts[0:5]
                l_type = int(l_parts.pop(1))
                if l_type == 1:
                    l_parts.append(str(CU_RADIUS))
                else:
                    l_parts.append(str(ZR_RADIUS))
                lammps_out_file_final.append(' '.join(l_parts))
                lammps_out_file_final.append('\n')
            with open(lammps_no_header_directory + '/' + 'no_header_' + filename, "w") as new_f:
                for line in lammps_out_file_final:
                    new_f.write(line)        

### Below cell is about seprating boundries from every lammps_step file and pass it through run_voro function to have the voroni analyze result in .vol file

In [10]:
lammps_directory = 'Lammps_Steps'
for filename in os.listdir(lammps_directory):
    if filename.endswith('.txt'):
        with open(os.path.join(lammps_directory, filename)) as f:
            lammps_out_file = f.read()
        boundaries = lammps_out_file.split("\n")[5:8]
        xmin, xmax = boundaries[0].split()
        ymin, ymax = boundaries[1].split()
        zmin, zmax = boundaries[2].split()
        run_voro(filename, float(xmin), float(xmax), float(ymin), float(ymax), float(zmin), float(zmax))

Container geometry        : [-4.68798:57.188] [-4.68798:57.188] [-4.68798:57.188]
Computational grid size   : 14 by 14 by 14 (estimated from file)
Filename                  : Lammps_Steps_No_Header/no_header_Lammps_Step_6.txt
Output string             : %i %s %A %v
Total imported particles  : 13500 (4.9 per grid block)
Total V. cells computed   : 13500
Total container volume    : 236901
Total V. cell volume      : 236901
Container geometry        : [-4.70204:57.202] [-4.70204:57.202] [-4.70204:57.202]
Computational grid size   : 14 by 14 by 14 (estimated from file)
Filename                  : Lammps_Steps_No_Header/no_header_Lammps_Step_10.txt
Output string             : %i %s %A %v
Total imported particles  : 13500 (4.9 per grid block)
Total V. cells computed   : 13500
Total container volume    : 237223
Total V. cell volume      : 237223
Container geometry        : [-4.70283:57.2028] [-4.70283:57.2028] [-4.70283:57.2028]
Computational grid size   : 14 by 14 by 14 (estimated from file)

### Below cell is about making voro++ output for each atom vectors with same length.

In [6]:
lammps_no_header_directory = 'Lammps_Steps_No_Header'
for filename in os.listdir(lammps_no_header_directory):
    if filename.startswith('no') and filename.endswith('.vol'):
        with open(os.path.join(lammps_no_header_directory, filename)) as f:
            lammps_out_file = f.readlines()
            lammps_out_file_final = []
            for l in lammps_out_file:
                l_parts = l.split(" ")
                l_parts_last = l_parts.copy()[-1]
                l_parts = l_parts[0:len(l_parts) - 1]
                for s in range(len(l_parts) - 1, 14):
                    l_parts.append(str(0))
                l_parts.append(l_parts_last.replace("\n", ''))
                lammps_out_file_final.append(' '.join(l_parts))
                lammps_out_file_final.append('\n')
            with open(lammps_no_header_directory + '/' + 'claer_' + filename, "w") as new_f:
                for line in lammps_out_file_final:
                    new_f.write(line)

### below cell is about making a whole dataframe for each step. first we calculate all distances from all atoms for every atom in the step file. then we sort the result list and append the IDs and their coordinatios. at the end we append every voroni index to dataframe using its result from the last cell.
## we can make a loop from it but it takes about 20 minuts for every step to procces with 13500 input atoms. so for better experince I think it's better to keep it like that and change the df 's input csv's index manually.

## PS:This version is about making little cube out of the sample with about 50 atoms on it.

In [5]:
import numpy as np

lammps_df = pd.read_csv('Lammps_Steps_No_Header/no_header_Lammps_Step_1.txt', sep=" ", header=None,)
face_order_df = pd.read_csv("Lammps_Steps_No_Header/claer_no_header_Lammps_Step_1.txt.vol", sep=" ", header=None,)
lammps = lammps_df.to_numpy()

little_lamps = np.array(lammps, copy=True)

some_l = []
for s in little_lamps:
    if s[1] > -5.5 and s[2] > -5.5 and s[3] > -3.45:
        some_l.append(int(s[0] - 1))

little_lamps = np.delete(little_lamps, some_l, axis=0)
print(len(little_lamps))


titles = ['id', 'x', 'y', 'z', 'type']


for i in range(1, 56):
    titles.append(str(i) + '-neighbour id')
    titles.append(str(i) + '-neighbour type')
    titles.append(str(i) + '-neighbour x')
    titles.append(str(i) + '-neighbour y')
    titles.append(str(i) + '-neighbour z')

titles.append('nb')
titles.append('1-faced')
titles.append('2-faced')
titles.append('3-faced')
titles.append('4-faced')
titles.append('5-faced')
titles.append('6-faced')
titles.append('7-faced')
titles.append('8-faced')
titles.append('9-faced')
titles.append('10-faced')
titles.append('11-faced')
titles.append('12-faced')
titles.append('13-faced')

list = []
for i in little_lamps:
    distance_dict = dict()
    coords_dict = dict
    for j in little_lamps:
        if i[0] == j[0]:
            pass
        else:
            distance_dict[j[0]] = calculate_distance(i[1], i[2], i[3], j[1], j[2], j[3])

    sorted_distance_dict = dict(sorted(distance_dict.items(), key=lambda item: item[1]))
    result = []


    for h in range(0, 55):
        id = int([*sorted_distance_dict][h])
        result.append(id)
        result.append(lammps_df[4].values[id - 1])
        result.append(lammps_df[1].values[id - 1])
        result.append(lammps_df[2].values[id - 1])
        result.append(lammps_df[3].values[id - 1])

    result.append(face_order_df[1].values[int(i[0]) - 1])
    result.append(face_order_df[2].values[int(i[0]) - 1])
    result.append(face_order_df[3].values[int(i[0]) - 1])
    result.append(face_order_df[4].values[int(i[0]) - 1])
    result.append(face_order_df[5].values[int(i[0]) - 1])
    result.append(face_order_df[6].values[int(i[0]) - 1])
    result.append(face_order_df[7].values[int(i[0]) - 1])
    result.append(face_order_df[8].values[int(i[0]) - 1])
    result.append(face_order_df[9].values[int(i[0]) - 1])
    result.append(face_order_df[10].values[int(i[0]) - 1])
    result.append(face_order_df[11].values[int(i[0]) - 1])
    result.append(face_order_df[12].values[int(i[0]) - 1])
    result.append(face_order_df[13].values[int(i[0]) - 1])
    result.append(face_order_df[14].values[int(i[0]) - 1])

    list.append(result)
    print(i[0])

little_lamps = np.hstack((little_lamps, list))
little_lamps = pd.DataFrame(little_lamps, columns=titles).to_csv('Output/output_little.csv', index=False)

56
185.0
454.0
651.0
1006.0
1009.0
1516.0
1590.0
1677.0
1956.0
2141.0
2814.0
3107.0
3160.0
3208.0
3219.0
3816.0
3971.0
4080.0
4236.0
4732.0
5119.0
5180.0
5464.0
5574.0
5660.0
5899.0
6108.0
6305.0
6441.0
6702.0
6794.0
6820.0
7321.0
7490.0
8081.0
8245.0
8451.0
8539.0
8697.0
8772.0
8910.0
8917.0
9391.0
10330.0
10670.0
11218.0
11635.0
12156.0
12507.0
12575.0
12624.0
12735.0
13122.0
13242.0
13337.0
13399.0
